# Text Classification
텍스트 문서의 다양한 분류방법에 대해 학습

In [52]:
import pandas as pd

In [12]:
df = pd.read_csv("sms_spam.csv")

In [13]:
df.head()

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...


In [18]:
categories = list(df['type'].values)
emails = list(df['text'].values)

### train set과 test set의 분리

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

적절한 비율로 train set과 test set을 분리하여 저장<br>
train set은 학습에 사용되고, test set은 검증에 사용<br>
default로 shuffle을 함: train set과 test set이 고르게 분포되도록 하기 위함

In [19]:
from sklearn.model_selection import train_test_split #sklearn에서 제공하는 split 함수를 사용
X_train, X_test, y_train, y_test = train_test_split(emails, categories, test_size=0.2, random_state=10)
# sklearn의 train_test_split 함수는 먼저 data set을 shuffle하고 주어진 비율에 따라 train set과 test set을 나눠 줌
# 위에서는 reviews를 X_train과 X_test로 8:2의 비율로 나누고, categories를 y_train과 y_test로 나눔
# 이 때 X와 y의 순서는 동일하게 유지해서 각 입력값과 label이 정확하게 match되도록 함
# random_state는 shuffle에서의 seed 값으로, 지정한 경우 항상 동일한 결과로 shuffle이 됨

print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

Train set count:  4447
Test set count:  1112


### TFIDF 변환

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
    
CountVectorizer로 Count Vector를 생성하고 TFIDF로 변환하는 대신, text로부터 직접 생성

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
#sklearn에서 제공하는 TfidfVectorizer를 이용
tfidf = TfidfVectorizer().fit(X_train) # X_train을 이용하여 vectorizer를 학습
tfidf #vectorize에서 사용한 매개변수 값들을 확인 -> 현재는 모두 default 값을 사용, 향후 tokenizer, max_features 등을 지정할 수 있음
# 상세한 매개변수 내용은 위 링크를 참조

TfidfVectorizer()

In [21]:
X_train_tfidf = tfidf.transform(X_train) #학습된 vectorizer를 이용하여 train set을 변환
X_train_tfidf.shape # 1600 (review 수) x 36310 (전체 corpus에서 사용된 단어의 수) 크기로 vector set이 생성됨
# matrix 안의 값은 해당 tfidf score임

(4447, 7727)

In [22]:
tfidf = TfidfVectorizer(max_features=2000).fit(X_train) #사용된 단어의 수가 너무 많은 경우, max_feature를 제한하여 학습이 가능
tfidf #max_factures 값이 사용된 것을 볼 수 있음

TfidfVectorizer(max_features=2000)

In [23]:
X_train_tfidf = tfidf.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_tfidf.shape) # 36310 대신 2000이 된 것을 확인
X_test_tfidf = tfidf.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_tfidf.shape)

Train set dimension: (4447, 2000)
Test set dimension: (1112, 2000)


## Naive Bayse Classifier (Scikit)

http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [24]:
#나이브 베이즈는 word count를 사용하므로 tfdif가 아닌 count vectorizer를 사용하여 학습 및 변환
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=2000).fit(X_train) #tfidf와 동일하게 max_feature를 제한하여 학습
X_train_cv = cv.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_cv.shape) # 36310 대신 2000이 된 것을 확인
X_test_cv = cv.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_cv.shape)

Train set dimension: (4447, 2000)
Test set dimension: (1112, 2000)


In [25]:
from sklearn.naive_bayes import MultinomialNB #sklearn이 제공하는 MultinomialNB 를 사용
NB_clf = MultinomialNB() # 분류기 선언

NB_clf.fit(X_train_cv, y_train) #train set을 이용하여 분류기(classifier)를 학습
#NB_clf.fit(X_train_tfidf, y_train) #tfidf 값을 사용할 수도 있으나, NB 이론에 맞지 않음

MultinomialNB()

In [26]:
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train))) #train set에 대한 예측정확도를 확인
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test))) #test set에 대한 예측정확도를 확인
#실제로 필요한 것은 test set에 대한 예측정확도이나, 과적합 (overfitting)의 문제가 있는지를 보기 위해 train set에 대한 예측정확도를 같이 확인
#print('Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train)))
#print('Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test)))

Train set score: 0.986
Test set score: 0.994


In [37]:
#여러 문장에 대해 count vectorier로 변환 후 학습된 분류기로 결과를 예측
print(NB_clf.predict(cv.transform(['the story was unimaginative', 'needs your URGENT collection.'])))

#위 첫째 문장에서 story를 actor로 바꿔서 예측
print(NB_clf.predict(cv.transform(['the actor was unimaginative'])))
#동일한 형용사라도 대상에 따라 결과가 바뀔 수 있음

['ham' 'spam']
['ham']


## Logistic Regression (Scikit)

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

예측하고자 하는 값 혹은 label이 연속적인 값이 아니고 분류(class)일 때 사용하는 regression 방법<br>
분류는 binary인 경우와 multi-class인 경우가 있음<br>
지금은 positive와 negative 두 class 중에서 선택하므로 binary classification 문제임

In [30]:
from sklearn.linear_model import LogisticRegression #sklearn이 제공하는 logistic regression을 사용

#count vector에 대해 regression을 해서 NB와 비교
LR_clf_cv = LogisticRegression() #분류기 선언
LR_clf_cv.fit(X_train_cv, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(LR_clf_cv.score(X_train_cv, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(LR_clf_cv.score(X_test_cv, y_test))) # test data에 대한 예측정확도
# count vector를 이용한 regression 결과가 tfidf보다 더 좋게 나옴
# 보통은 tfidf가 더 좋은 결과를 보이는데, 이와 같이 상황에 따라 다른 결과가 나오기도 함
# 지금은 train data의 수가 1,600개인데 비해, 추정해야 하는 parameter의 수가 2,000개로 sample 수가 학습에 부족한 상황, 
# 따라서 예상 못한 다양한 결과가 나올 수 있음
# 좀더 data가 많은 상황에서의 test가 필요

Train set score: 0.995
Test set score: 0.984


In [31]:
#tfidf vector를 이용해서 분류기 학습
LR_clf = LogisticRegression() #분류기 선언
LR_clf.fit(X_train_tfidf, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test))) # test data에 대한 예측정확도
# NB에 비해 더 좋은 결과가 나오는 것을 확인

Train set score: 0.978
Test set score: 0.979


In [40]:
# NB 분류기에서 사용했던 예제로 결과 확인, 실제로 결과가 더 나아졌음을 확인할 수 있음
LR_clf.predict(tfidf.transform(['the story was unimaginative', 'complimentary 4 STAR Ibiza Holiday or £10,000 cash needs your URGENT collection. 09066364349 NOW from Landline not to lose out! Box434SK38WP150PPM18+', 'kate winslet is accessible']))

array(['ham', 'spam', 'ham'], dtype='<U4')

### Ridge regression
https://brilliant.org/wiki/ridge-regression/
Tikhonov Regularization, colloquially known as **ridge regression**, is the most commonly used regression algorithm to approximate an answer for an equation with no unique solution. This type of problem is very common in machine learning tasks, where the "best" solution must be chosen using limited data.

https://en.wikipedia.org/wiki/Tikhonov_regularization
the most commonly used method of **regularization** of ill-posed problems. In statistics, the method is known as **ridge regression**, in machine learning it is known as **weight decay**, and with multiple independent discoveries.
![](https://ds055uzetaobb.cloudfront.net/image_optimizer/37ea8b78480bb520a3e30b7113689f7b77cecc2d.png)
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

Ridge regression은 과적합을 방지하기 위해 사용됨<br>
과적합은 분류기가 training data에 지나치게 fitting되어 실제 상황 혹은 test data에서는 좋은 성능이 나타나지 않는 상황을 말함<br>
위 그림에서 파란색의 선은 보다 간단한 초록색의 선으로 주어진 데이터를 설명할 수 있음에도 불구하고, 훨씬 복잡한 곡선으로 fitting이 되어 있으며, 이로 인해 training data에 대한 예측정확도는 높으나 test data에 대해서는 예측정확도가 떨어짐<br>
이와 같은 현상을 방지하기 위해서는 곡선이 지나치게 복잡해지지 않도록 parameter를 억제하면 됨<br>
Rigde regression은 parameter에 대해 제약을 줌으로써 학습과정에서 parameter가 과도하게 변하지 않도록 함<br>

In [41]:
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier() #릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) #학습
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))
# 일반적으로 ridge regression을 쓰면 쓰지 않은 경우보다 train data에 대한 예측정확도는 떨어지고 test data는 올라가게 됨
# 여기서는 train set에 대한 예측정확도가 같이 상승하는 진귀한 경우가 발생

Train set score: 0.992
Test set score: 0.991


### Lasso regression for feature selection
https://en.wikipedia.org/wiki/Lasso_(statistics)
In statistics and machine learning, **lasso** (least absolute shrinkage and selection operator) (also Lasso or LASSO) is a regression analysis method that performs both **variable selection** and **regularization** in order to enhance the prediction accuracy and **interpretability** of the statistical model it produces. 

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

Lasso regression은 ridge regression과 비슷하게 parameter에 제약을 주지만, 0에 가까운 parameter를 완전히 0으로 바꿔서 결과적으로 feature를 선택하게 된다는 차이가 있음

In [42]:
from sklearn.linear_model import LogisticRegression
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear') # Lasso는 동일한 LogisticRegression을 사용하면서 매개변수로 지정
lasso_clf.fit(X_train_tfidf, y_train) # train data로 학습
print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1]) 
# parameter 혹은 coefficient 중에서 0이 아닌 것들의 개수를 출력
# 2000개 중에서 78개만 선택된 것을 볼 수 있음
# 예측률은 rigde나 일반 logistic에 비해 떨어지지만, 실제로 영향을 미치는 단어들이 어떤 것들인지 확인할 수 있다는 장점이 있음

Train set score: 0.974
Test set score: 0.978
Used features count: 115 out of 2000


In [43]:
print(len(tfidf.vocabulary_)) # tfidf에 사용된 단어의 수
tfidf_voca = tfidf.get_feature_names() # tfidf에서 단어이름을 가져옴
tfidf_voca[:10] # 앞 10개를 출력

2000


['00',
 '000',
 '02',
 '03',
 '04',
 '06',
 '0800',
 '08000839402',
 '08000930705',
 '08001950382']

In [44]:
print((lasso_clf.coef_ != 0)[0].tolist()[:100]) 
# lasso에 사용된 단어들 중 coefficient의 사용여부를 리스트로 변환하여 앞부터 100개를 출력해 봄

[False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, True, True, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [45]:
selected_features = []
for i, sign in enumerate((lasso_clf.coef_ != 0)[0].tolist()):
    if sign: selected_features.append(tfidf_voca[i]) #사용여부가 True인 단어들만 selected_features에 저장

In [46]:
print(selected_features) #78개의 선택된 단어들을 출력 - 즉 positive, negative에 결정적 영향을 미치는 단어들
len(selected_features)

['0800', '100', '10p', '150p', '16', '18', '2003', '20p', '50', '500', '800', 'am', 'at', 'auction', 'award', 'awarded', 'bid', 'but', 'call', 'calls', 'camera', 'can', 'cant', 'cash', 'chat', 'claim', 'code', 'collection', 'com', 'come', 'cost', 'customer', 'da', 'enjoy', 'for', 'free', 'from', 'games', 'going', 'got', 'have', 'he', 'help', 'him', 'home', 'how', 'http', 'if', 'in', 'it', 'its', 'later', 'latest', 'll', 'ltd', 'me', 'message', 'min', 'mob', 'mobile', 'msg', 'my', 'new', 'nokia', 'now', 'of', 'ok', 'on', 'or', 'order', 'our', 'out', 'per', 'play', 'po', 'private', 'prize', 'rate', 'receive', 'reply', 'ringtone', 'selected', 'send', 'service', 'services', 'sexy', 'sky', 'sms', 'so', 'sorry', 'special', 'statement', 'stop', 'text', 'that', 'then', 'to', 'tones', 'txt', 'uk', 'ur', 'urgent', 'video', 'voucher', 'wap', 'we', 'week', 'what', 'who', 'win', 'with', 'won', 'www', 'you', 'your']


115

In [47]:
tfidf_voca[(lasso_clf.coef_ != 0)[0].tolist().index(True)]

'0800'

In [48]:
print([tfidf_voca[i] for i, j in enumerate((lasso_clf.coef_ != 0)[0].tolist()) if j]) #선택된 단어들을 출력하는 또다른 방법

['0800', '100', '10p', '150p', '16', '18', '2003', '20p', '50', '500', '800', 'am', 'at', 'auction', 'award', 'awarded', 'bid', 'but', 'call', 'calls', 'camera', 'can', 'cant', 'cash', 'chat', 'claim', 'code', 'collection', 'com', 'come', 'cost', 'customer', 'da', 'enjoy', 'for', 'free', 'from', 'games', 'going', 'got', 'have', 'he', 'help', 'him', 'home', 'how', 'http', 'if', 'in', 'it', 'its', 'later', 'latest', 'll', 'ltd', 'me', 'message', 'min', 'mob', 'mobile', 'msg', 'my', 'new', 'nokia', 'now', 'of', 'ok', 'on', 'or', 'order', 'our', 'out', 'per', 'play', 'po', 'private', 'prize', 'rate', 'receive', 'reply', 'ringtone', 'selected', 'send', 'service', 'services', 'sexy', 'sky', 'sms', 'so', 'sorry', 'special', 'statement', 'stop', 'text', 'that', 'then', 'to', 'tones', 'txt', 'uk', 'ur', 'urgent', 'video', 'voucher', 'wap', 'we', 'week', 'what', 'who', 'win', 'with', 'won', 'www', 'you', 'your']


### Dimension을 줄이는 또다른 방법: SVD
lasso가 feature selection이라면, SVD는 feature extraction

**Feature selection** (https://en.wikipedia.org/wiki/Feature_selection) 

In machine learning and statistics, feature selection, also known as variable selection, attribute selection or variable subset selection, is the process of selecting a subset of relevant features (variables, predictors) for use in model construction. Feature selection techniques are used for four reasons:

* simplification of models to make them easier to interpret by researchers/users,
* shorter training times,
* to avoid the curse of dimensionality,
* enhanced generalization by reducing overfitting (formally, reduction of variance)

Text mining에서는 단어의 수를 다양한 방법으로 줄일 수 있음

* 전처리 과정에서 의미 없는 단어 삭제
* stopwords 삭제
* 빈도가 낮은 단어 삭제
* 빈도가 높은 단어 삭제
* 결과에 영향을 미치지 않는 단어 삭제

**Feature extraction**: (https://en.wikipedia.org/wiki/Feature_extraction)

In machine learning, pattern recognition and in image processing, feature extraction starts from an initial set of measured data and builds derived values (features) intended to be informative and non-redundant, facilitating the subsequent learning and generalization steps, and in some cases leading to better human interpretations. Feature extraction is related to dimensionality reduction.

* PCA(Principal component analysis)
* LSA(Latent semantic analysis)
* SVD(Singular-value decomposition)

### PCA(Principal component analysis)

https://en.wikipedia.org/wiki/Principal_component_analysis

**Principal component analysis (PCA)** is a statistical procedure that uses an *orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables* called principal components. If there are {\displaystyle n} n observations with {\displaystyle p} p variables, then the number of distinct principal components is {\displaystyle \min(n-1,p)} {\displaystyle \min(n-1,p)}. This transformation is defined in such a way that the first principal component has the *largest possible variance* (that is, accounts for as much of the variability in the data as possible), and each succeeding component in turn has the highest variance possible under the constraint that it is orthogonal to the preceding components. The resulting vectors are an uncorrelated orthogonal basis set. PCA is sensitive to the relative scaling of the original variables.

**PCA(주성분분석)**: 데이터의 분산을 최대한 보존하는 새로운 축을 찾아 변환함으로써 차원을 축소

https://stats.stackexchange.com/questions/2691/making-sense-of-principal-component-analysis-eigenvectors-eigenvalues

![](g1.png)
![](pca1.gif)
![](pca2.gif)

### LSA(Latent Semantic Analysis)

**Latent semantic analysis (LSA)** is a technique in natural language processing, in particular distributional semantics, of analyzing relationships between a set of documents and the terms they contain by producing a set of concepts related to the documents and terms. LSA assumes that *words that are close in meaning will occur in similar pieces of text* (the distributional hypothesis). A matrix containing word counts per paragraph (rows represent unique words and columns represent each paragraph) is constructed from a large piece of text and a mathematical technique called **singular value decomposition (SVD)** is used to reduce the number of rows while preserving the similarity structure among columns. Words are then compared by taking the cosine of the angle between the two vectors (or the dot product between the normalizations of the two vectors) formed by any two rows. Values close to 1 represent very similar words while values close to 0 represent very dissimilar words.

**SVD(특이값 분해)** https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/04/06/pcasvdlsa/
"*truncated SVD*는 Σ 행렬의 대각원소(특이값) 가운데 상위 t개만 골라낸 형태. 이렇게 하면 행렬 A를 원복할 수 없게 되지만, 데이터 정보를 상당히 압축했음에도 행렬 A를 근사할 수 있음." **잠재의미분석(LSA)** 은 바로 이 방법을 사용"

![](tSVD.png)

동영상 확인: https://en.wikipedia.org/wiki/Latent_semantic_analysis

### Scikit-learn LSA
http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html
<br>
**Dimensionality reduction using truncated SVD (aka LSA).**

This transformer performs linear dimensionality reduction by means of truncated singular value decomposition (SVD). Contrary to PCA, this estimator does not center the data before computing the singular value decomposition. This means it can work with scipy.sparse matrices efficiently.

In [49]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42) #압축할 component의 수 지정
svd.fit(X_train_tfidf)  # train data로 학습 -> unsupervised이므로 y가 필요 없음
print(svd.explained_variance_ratio_)  #계산된 각 component가 설명하는 분산의 비율
print(svd.explained_variance_ratio_.sum())  #선택된 component들이 설명하는 분산의 합 -> 선택한 component의 수에 따라 달라짐
# 현재 결과에서는 100개의 선택된 component가 전체 분산의 34% 정도를 설명하고 있음
print(svd.singular_values_) #위 그림에서 오른쪽 가운데 대각행렬의 값
print(svd.components_.shape) # 위 그림에서 가장 오른쪽 행렬의 값

[0.00707897 0.0100556  0.00985463 0.00940513 0.00777477 0.00695628
 0.0061751  0.00602694 0.00581524 0.00556494 0.0054602  0.00539744
 0.00516919 0.00511072 0.00494922 0.00492746 0.00479858 0.00462921
 0.00451675 0.00446954 0.00442049 0.00436371 0.00426803 0.00422459
 0.00419024 0.00411059 0.00406554 0.00400562 0.00395285 0.00392338
 0.00386094 0.00382293 0.00369616 0.00367866 0.00361268 0.00357086
 0.00354793 0.00349725 0.00344584 0.00341227 0.00338689 0.00332727
 0.00331214 0.00329053 0.00326242 0.003195   0.00317421 0.00311967
 0.00309303 0.00307338 0.00304898 0.00301602 0.00298161 0.00296695
 0.0029405  0.00292531 0.0028806  0.00287243 0.00284383 0.00281694
 0.00277088 0.00274432 0.00272697 0.00272067 0.00265578 0.00265135
 0.00261825 0.00258976 0.00258252 0.00257865 0.00255722 0.00248049
 0.00246693 0.00244716 0.0024246  0.00240258 0.00238522 0.00236765
 0.00232812 0.00231633 0.00230022 0.00227767 0.00227013 0.00223568
 0.00221793 0.00219076 0.00216925 0.00214988 0.00214727 0.0021

In [50]:
X_train_svd = svd.transform(X_train_tfidf) #선택된 component를 이용하여 2,000개의 feature로부터 feature extract (dimension reduce)
print(X_train_tfidf.shape) #축소하기 전의 차원
print(X_train_svd.shape) #축소된 후의 차원, 2000 -> 100개로 줄어 있음

(4447, 2000)
(4447, 100)


#### LSA를 이용한 Logistic Regression

In [51]:
X_test_svd = svd.transform(X_test_tfidf)

from sklearn.linear_model import LogisticRegression
SVD_clf = LogisticRegression()
SVD_clf.fit(X_train_svd, y_train) #축소된 값으로 학습
print('Train set score: {:.3f}'.format(SVD_clf.score(X_train_svd, y_train)))
print('Test set score: {:.3f}'.format(SVD_clf.score(X_test_svd, y_test)))
# NB, Lasso보다 좋지만 Ridge, 일반보다는 나쁜 값
# 상황에 따라 달라질 수 있음

Train set score: 0.963
Test set score: 0.970
